# 笔记本说明：
本笔记本的作用为批量给图片做标注并进行后续数据处理。

## 第一部分使用步骤：
1\. 确认数据所在文件夹，并将数据总文件夹的绝对路径填入`PATH`变量，并按shift+Enter运行（如果路径不存在，会警告）
> 如果有不需要处理的文件夹，需重命名为："-"+原名称，在后续步骤中，该文件夹中的数据会被忽视。故可以将不处理的数据和处理过的文件夹前面加上一个`"-"`   

> 实验数据的文件结构规定为：  
```
- data  
    |  
    -420v5(experiment级文件）    
        |  
        -***.hsv  
        -*.  
    -420v10q2  
    -600v1000q6  
    -etc.  
```    

2\. 运行第2个cell，查看并确认子文件夹名称是否是要处理的那些
- 如果是，可以跳过第3个cell中ignore_dir()和deignore_dir()函数不运行，或者向其中传入-1（-1表示不处理）并运行
- 如果有需要忽略的文件夹，向函数中输入子文件夹名称对应的序号，并运行。运行后回到上一cell

3\. 根据第2个3个cell的调整初始化并记录子文件夹名字，根据HSV文件名批量新建子文件夹（此后请将HSV文件生产的JPEG图片列表保存到对应的子文件夹中）


需要添加的内容：
根据需要改写取点函数，给定需求，1000个点
补充验证条件，必须对角线取点
图片名称序号问题。。。

In [7]:
PATH = r"E:\zhouhan"
# 填入文件名前缀，最好为实验的描述，该描述将作为后续所有数据文件名的前缀
# 如要获取data_600V350Q401 - data_600V350Q440的所有csv文件，
# 填入600V350Q4
PREFIX = "TEST"

In [8]:
# ********************************
# DO NOT CHANGE ANYTHING BELOW !!!
# ********************************
from PIL import Image
import numpy as np
import pandas as pd
import os
import glob
import time
import itertools
import matplotlib
matplotlib.use('TkAgg')   # 允许GUI式matplotlib，即有弹窗. it must be set before import pyplot
import matplotlib.pyplot as plt

assert os.path.exists(PATH)  # 必须填入有效的起始文件夹data的绝对路径名称
items = os.listdir(PATH)  # PATH下的所有item
dirs = [item for item in items if os.path.isdir(os.path.join(PATH, item))] # 所有item中的所有文件夹
print("------list of directories------")
for i,j  in enumerate(dirs):
    print(str(i).rjust(2) + ":" + str(j).rjust(12))

def ignore_dir(index, lst=dirs):
    if index==-1:
        return None
    elif index < len(lst) and lst[index][0] != '-' :  # 已经忽略的不重复忽略
        os.rename(PATH+"\\"+lst[index], PATH+"\\-"+lst[index])  
    return None

def deignore_dir(index, lst=dirs):
    if index==-1:
        return None
    elif index < len(lst) and lst[index][0] == '-':  # 已经未忽略的不重复未忽略
        os.rename(PATH+"\\"+lst[index], PATH+"\\"+lst[index][1:])
    return None

------list of directories------
 0:        -420
 1:   420V100Q4
 2:    420V10Q2
 3:   420V20Q02
 4:       420V5
 5:  600V1000Q6
 6:  600V350 Q4
 7:   600V500Q4


In [9]:
ignore_dir(-1)
deignore_dir(-1)

In [10]:
# ********************************
# DO NOT CHANGE ANYTHING BELOW !!!
# ********************************

### 获取目录和子目录（如果没有则根据目录中的*.HSV文件数量批量创建子文件夹）

items = os.listdir(PATH)  # PATH下的所有item
dirs = [item for item in items if os.path.isdir(os.path.join(PATH, item))] # 所有item中的所有文件夹
dirs = [dir for dir in dirs if not dir[0]=="-"]  # 忽略不处理的数据文件夹（需要忽略的实验文件名前加"-"）

# 改名，将所有experiments级文件夹名由小写改为大写并重新获取目录
for dir in dirs:
    if dir.islower():
        os.rename(PATH+"\\"+dir, PATH+"\\"+dir.upper())

items = os.listdir(PATH)  # PATH下的所有item
dirs = [item for item in items if os.path.isdir(os.path.join(PATH, item))] # 所有item中的所有文件夹
dirs = [dir for dir in dirs if not dir[0]=="-"]  # 忽略不处理的数据文件夹（需要忽略的实验文件前加"-"）
print("***文件名标准化完成***")

# 如果没有创建 子文件夹 的话，按照.hsv文件的名称创建，并以列表的字典形式{dir: list_subdir}记录所有subdirs
subdirs = {}
for dir in dirs:
    list_hsv = glob.glob(PATH+'\\'+dir+'\\*.hsv') # 列出该文件夹下所有hsv文件的路径
    list_subdir = [s.split('.')[0].split("\\")[-1] for s in list_hsv] # 列出该文件夹下所有hsv文件的名字，注意，不包括后缀.hsv
    subdirs[dir] = list_subdir  # 更新subdirs词典

    # 逐文件夹新建和.hsv文件同名的子文件夹
    for subdir in list_subdir:
        if not os.path.exists(PATH+"\\"+dir+"\\"+subdir):
            os.mkdir(PATH+"\\"+dir+"\\"+subdir) 

print("***子文件夹创建完成***")
print("***目录和子目录名称已获取完成***")
print("------list of directories------")
for i,j  in enumerate(dirs):
    print(str(i).rjust(2) + ":" + str(j).rjust(12))

***文件名标准化完成***
***子文件夹创建完成***
***目录和子目录名称已获取完成***
------list of directories------
 0:   420V100Q4
 1:    420V10Q2
 2:   420V20Q02
 3:       420V5
 4:  600V1000Q6
 5:  600V350 Q4
 6:   600V500Q4


## 第二部分：统计数密度分布函数使用方法
1\. 运行第一个cell  
2\. 在之后的cell中输入函数 mark_image(idir, isubdir, iimg=0, stop=3767, step=1)
例如 键入mark_image(5, 0, iimg=0,stop=3765,step=300)并运行表示，要标注序号为5的（第6个）文件夹的序号为0（第1个）的子文件夹，从iimg=0（第1个）开始，到stop=3765结束（不包括第3766张），步长为300。默认从第一张照片0标注到最后一张照片3767，步长为1。
该函数运行后会弹出一个图像窗口
- 在该窗口中，单击**左键取点**，点击**右键取消**最近一次取点，单击**中键完成**该图片。
    - 如果满足标注要求，会根据函数的参数自动切换到下一张，继续标注即可。  
        图片标注必须满足条件如下：（否则自动重新标注当前图片）  
        - 不能不标（防止误触中键）
        - 标注的点为4的倍数（每个液滴用且仅用4个点标注）
        - 4个点不能相距太远（目前设定为250个像素）
        > 标注圆圈的四个点必须按照对角的顺序进行标注，切记！此错误本程序无法检出。
    - 如果不满足标注要求，
- 图像标题会显示当前标注的内容，如果标注错误或者发现破碎等需要记录时可以手工记录下该标题备后续使用。
- 在笔记本中会显示取点的列表、标注状态和进度，可以关注一下

> 如果发现该函数的参数填写错误，可以点击笔记本上方的□（Interrupt the kernel）打断函数并重新运行即可（**不要关闭窗口**）  
> 注意！：该窗口在标注结束不再接受取点后，**不要关闭**，否则会导致程序出错，需要重新启动一次kernel再从头运行整个笔记本。  

标注结束后，窗口停止取点（再重申一次，**不要关闭窗口**），笔记本输出“所有数据保存完毕，程序退出”，数据保存为**csv格式**到**工作文件夹**，csv文件名和子文件夹相同。（如有问题可以通过记事本或者Excel打开该文件进行修改。）




In [24]:
# ********************************
# DO NOT CHANGE ANYTHING BELOW !!!
# ********************************
def isvalid(pts):
    # 如果pts是空的
    if len(pts) == 0:
        return False
    # 如果不满足 正好4个一组
    if np.mod(len(pts), 4) != 0:
        return False
    # 如果每组四个点两两之间的距离过大（大于250）
    for start, stop in zip(range(0, len(pts)-1, 4), range(4, len(pts)+1, 4)): 
        for a, b in itertools.combinations(pts[start: stop], 2): 
            if (abs(a[0] - b[0]) > 250 or abs(a[1] - b[1]) > 250):
                return False
    if False:# 如果没有按照对角线顺序选取
        return False
#     if s=input("Sure to sumbit?") # 用户确认功能
    else:
        return True

#iimg 为图片的编号，如果你想从0005号图片开始标注，请输入4，可以理解为跳过前4张图片
def mark_image(idir, isubdir, iimg=0, stop=3767, step = 1, sample = None):
    img_paths = glob.glob(PATH+"\\"+dirs[idir]+"\\"+subdirs[dirs[idir]][isubdir]+"\\*.jpg")
    # 如果输入参数没有什么错误，就进行切片
    if ((idir in range(len(dirs))) 
        and (isubdir in range(len(subdirs[dirs[idir]]))) 
        and (iimg not in range(3767))
        and (stop not in range(3767))):
        print("Invalid input: \n(index of image must range between 0 - 3766)\n")
        return
    img_paths = img_paths[iimg:stop:step]
    if img_paths == []:
        return
    
    print("开始标注%s中的图片" % dirs[idir]+subdirs[dirs[idir]][isubdir])
    
    if(os.path.exists(os.getcwd()+"\\data_" + str(subdirs[dirs[idir]][isubdir]) + ".csv")):
        print("Note that there already exist data, please check whether to remove")
    
    for img_index, img_path in zip(range(iimg,stop,step), img_paths):
        name_experiment = img_path.split(".")[0].split("\\")[-1].split("_")[0]
        name_pic = img_path.split(".")[0].split(" / ")[-1].split("_")[-1]
        print("正在标注%s:" % name_pic)
        im = Image.open(img_path)
        plt.imshow(im)
        grid_X,grid_Y =  zip(*[(200,100),(200,600), (200,100),(1000,100),
                               (200,350),(1000,350),(400,100),(400,600),
                               (600,100),(600,600), (800,100),(800,600)])
        for n_grid in range(min(len(grid_X),len(grid_Y)) // 2):
            plt.plot(grid_X[2*n_grid: 2*n_grid+2], grid_Y[2*n_grid: 2*n_grid+2],'k--',lw=1)
        grid_X,grid_Y =  zip(*[(1000,100),(1000,600),(200,600),(1000,600)])
        for n_grid in range(min(len(grid_X),len(grid_Y)) // 2):
            plt.plot(grid_X[2*n_grid: 2*n_grid+2], grid_Y[2*n_grid: 2*n_grid+2],'k-',lw=1)
        
        plt.title(name_experiment + ": " + str(idir) + " / " + str(isubdir) + " / " + name_pic)

        pts = []
        pts = plt.ginput(n=-1, timeout=0, show_clicks=True)

        
        # 如果数据不合格就重新标注一遍
        if not isvalid(pts):
            print("重新标注%s" % name_pic)
            pts = []
            mark_image(idir, isubdir, iimg=img_index, stop=img_index+1)
            continue
        
        print("完成标注 %s:" %name_pic)
        # 保存数据
        n_circle = int(len(pts)/4)
        circle = []
        for i in range(n_circle):
            circle += [i] * 4
        df_circle = pd.DataFrame(circle, columns=['circle'])
        df_dir = pd.DataFrame({'dir':[dirs[idir]]*len(pts),
                               'subdir': [subdirs[dirs[idir]][isubdir]]*len(pts), 
                               "timestamp":[time.asctime()]*len(pts),
                               "pic":name_pic})
        df_pts = pd.DataFrame(pts, columns=list("xy"), copy=True)
        df_data = pd.concat([df_pts, df_circle,df_dir], axis=1)
        print(df_data)
        if not os.path.exists("NDD"):
            os.mkdir('NDD')
#         if 'NDD' not in os.path.dirname(os.getcwd()):
#             os.mkdir('NDD')    
        df_data.to_csv(os.getcwd()+"\\NDD\\data_" + str(subdirs[dirs[idir]][isubdir]) + ".csv",mode='a', header=False)
        print("数据保存完毕")
    print("所有数据保存完毕，程序退出")
    plt.close()
    return None

In [25]:
mark_image(5, 0, iimg=0,stop=1000,step=300)

开始标注600V350 Q4中的图片600V350Q401
Note that there already exist data, please check whether to remove
正在标注0001:


C:\ProgramData\Anaconda3\envs\stat\lib\site-packages\matplotlib\backend_bases.py:2453: MatplotlibDeprecationWarning: Using default event loop until function specific to this GUI is implemented
  warnings.warn(str, mplDeprecation)


完成标注 0001:
            x           y  circle         dir   pic       subdir  \
0  487.570433  409.648501       0  600V350 Q4  0001  600V350Q401   
1  525.563382  440.335114       0  600V350 Q4  0001  600V350Q401   
2  491.954235  438.873847       0  600V350 Q4  0001  600V350Q401   
3  516.795779  400.880898       0  600V350 Q4  0001  600V350Q401   

                  timestamp  
0  Sat Mar 24 14:52:15 2018  
1  Sat Mar 24 14:52:15 2018  
2  Sat Mar 24 14:52:15 2018  
3  Sat Mar 24 14:52:15 2018  
数据保存完毕
正在标注0301:
完成标注 0301:
            x           y  circle         dir   pic       subdir  \
0  484.647899  412.571036       0  600V350 Q4  0301  600V350Q401   
1  481.725364  447.641451       0  600V350 Q4  0301  600V350Q401   
2  500.721839  427.183709       0  600V350 Q4  0301  600V350Q401   
3  459.806355  430.106243       0  600V350 Q4  0301  600V350Q401   

                  timestamp  
0  Sat Mar 24 14:52:19 2018  
1  Sat Mar 24 14:52:19 2018  
2  Sat Mar 24 14:52:19 2018  
3  Sat Ma

In [12]:
mark_image(5, 1, iimg=0,stop=3765,step=300)

In [13]:
mark_image(5, 2, iimg=0,stop=3765,step=300)

In [14]:
mark_image(5, 3, iimg=0,stop=3765,step=300)

In [ ]:
mark_image(5, 4, iimg=0,stop=3765,step=300)

In [10]:
mark_image(5, 5, iimg=0,stop=3765,step=300)

In [10]:
points = [(1,1),(3,3),(1,1),(3,4)]
   
# plt.plot(x,y,"+")
# plt.show()

# 如果每组四个点对应的两条线段相交
def isintersected(pts):
    assert(len(pts) == 4)
    def isseperated(pts):
        """如果p1,p2在P3,p4直线的两边，则返回True，否则返回False"""
        p1,p2,p3,p4 = pts
        # 先求斜率a，注意x1-x2!=0时，直接用x坐标进行比较
        if abs(p1[0]-p2[0]) < 1e-6:
            if abs(p1[1]-p2[1]) < 1e-6:
                return False
            elif (p3[0]-p1[0]) * (p4[0]-p1[0]) < 0:
                return True
            else:
                return False
        elif abs(p1[1]-p2[1]) < 1e-6:
            if (p3[0]-p1[0]) * (p4[0]-p1[0]) < 0:
                return True
            else:
                return False
        else:
            a = (p1[1]-p2[1])/(p1[0]-p2[0])
            b = p1[1] - a * p1[0]
            if not ((p3[1] > a * p3[1] + b) and (p4[1] > a * p4[1] + b)):
                return True
            return False
        
    return (isseperated(pts) and isseperated(reversed(pts)))



In [11]:
isintersected(points)

True

合并所有"data_\*.csv"为data.csv文件，并计算出每个点的面积和


In [10]:
glob.glob(os.getcwd()+"\\"+"data_*.csv")

['C:\\Users\\Jerry\\Documents\\jupyter\\ImageAnalysis\\data_420V100Q401.csv']

In [14]:
HEADER = ['index','x', 'y', 'circle', 'dir', 'pic', 'subdir', 'timestamp']
data_total = pd.DataFrame(columns=HEADER)
print(data_total)
for data_path in glob.glob(os.getcwd()+"\\"+"data_*.csv"):
    data_patch = pd.read_csv(data_path, header=None, names=HEADER)
    print("***" + data_path.split("\\")[-1] + " has been added! ***\n")
#     print("data_patch:\n", data_patch.head(), "\n\n")
    data_total = pd.concat([data_total, data_patch])
print("*** All data has been merged! ***\n------ It looks like this ------\n\n", data_total, "\n......\n\n------ Here shows some description ------", data_total.describe())

Empty DataFrame
Columns: [index, x, y, circle, dir, pic, subdir, timestamp]
Index: []
***data_420V100Q401.csv has been added! ***

*** All data has been merged! ***
------ It looks like this ------

    index           x           y circle        dir   pic       subdir  \
0      0  428.385306  295.816323      0  420V100Q4  3766  420V100Q401   
1      1  499.194298  375.637369      0  420V100Q4  3766  420V100Q401   
2      2  524.943023  297.103759      0  420V100Q4  3766  420V100Q401   
3      3  403.924017  364.050443      0  420V100Q4  3766  420V100Q401   
4      4  705.184095  113.000378      1  420V100Q4  3766  420V100Q401   
5      5  739.944874  149.048593      1  420V100Q4  3766  420V100Q401   
6      6  742.519746  116.862687      1  420V100Q4  3766  420V100Q401   
7      7  702.609223  141.323975      1  420V100Q4  3766  420V100Q401   
8      0  517.218406  254.618364      0  420V100Q4  3767  420V100Q401   
9      1  549.404311  300.966068      0  420V100Q4  3767  420V100Q401 

In [15]:
data = data_total.copy()

In [27]:
a = 3
b = 4
(lambda x:np.hypot(x[0],x[1]))((a,b))

27

In [52]:
# x = [3,0,3,6]
# y = [4,0,8,4]
# df_x = pd.Series(x)
# print(df_x)
# print(df_x.values)
# laxis = np.hypot(abs(x[0]-x[1]), abs(y[0]-y[1]))
# saxis = np.hypot(abs(x[2]-x[3]), abs(y[2]-y[3]))
# get_geo_mean = lambda x: np.sqrt(x[0]*x[1])
# geo_mean = get_geo_mean((laxis, saxis))
# geo_mean

0    3
1    0
2    3
3    6
dtype: int64
[3 0 3 6]


5.0

In [46]:
data

,index,x,y,circle,dir,pic,subdir,timestamp
0,0,428.385306,295.816323,0,420V100Q4,3766,420V100Q401,Thu Mar 15 15:23:15 2018
1,1,499.194298,375.637369,0,420V100Q4,3766,420V100Q401,Thu Mar 15 15:23:15 2018
2,2,524.943023,297.103759,0,420V100Q4,3766,420V100Q401,Thu Mar 15 15:23:15 2018
3,3,403.924017,364.050443,0,420V100Q4,3766,420V100Q401,Thu Mar 15 15:23:15 2018
4,4,705.184095,113.000378,1,420V100Q4,3766,420V100Q401,Thu Mar 15 15:23:15 2018
5,5,739.944874,149.048593,1,420V100Q4,3766,420V100Q401,Thu Mar 15 15:23:15 2018
6,6,742.519746,116.862687,1,420V100Q4,3766,420V100Q401,Thu Mar 15 15:23:15 2018
7,7,702.609223,141.323975,1,420V100Q4,3766,420V100Q401,Thu Mar 15 15:23:15 2018
8,0,517.218406,254.618364,0,420V100Q4,3767,420V100Q401,Thu Mar 15 15:24:02 2018
9,1,549.404311,300.966068,0,420V100Q4,3767,420V100Q401,Thu Mar 15 15:24:02 2018


In [60]:
# print(data[['x','y']])
# data[['dir','subdir','circle','x','y']].groupby(by=["dir", "subdir", "circle"]).apply(lambda x:x.sum())
def get_d_equiv(df):
    print("\n\n------------------\n")
    print(df)
    x = df['x'].values
    y = df['y'].values
    print(x,"\n")
    print(y,"\n")
#     x = [3,0,3,6]
#     y = [4,0,8,4]
    laxis = np.hypot(abs(x[0]-x[1]), abs(y[0]-y[1]))
    saxis = np.hypot(abs(x[2]-x[3]), abs(y[2]-y[3]))
    get_geo_mean = lambda x: np.sqrt(x[0]*x[1])
    print(get_geo_mean((laxis, saxis)))
    return get_geo_mean((laxis, saxis))
#     return pd.DataFrame({'original' : group,
#                  'demeaned' : group - group.mean()})
# np.hypot()
data = pd.read_csv(os.getcwd()+"\\data_420V100Q401.csv",names=HEADER)
data_analyzed = data.groupby(by=["dir", "subdir", 'pic', "circle"])[['x','y']].apply(get_d_equiv)
data_analyzed.to_csv('aaa')



------------------

            x           y
0  428.385306  295.816323
1  499.194298  375.637369
2  524.943023  297.103759
3  403.924017  364.050443
[428.38530552 499.19429827 524.9430229  403.92401712] 

[295.81632308 375.63736945 297.10375931 364.05044336] 

121.4788182390552


------------------

            x           y
0  428.385306  295.816323
1  499.194298  375.637369
2  524.943023  297.103759
3  403.924017  364.050443
[428.38530552 499.19429827 524.9430229  403.92401712] 

[295.81632308 375.63736945 297.10375931 364.05044336] 

121.4788182390552


------------------

            x           y
4  705.184095  113.000378
5  739.944874  149.048593
6  742.519746  116.862687
7  702.609223  141.323975
[705.18409533 739.94487359 742.51974605 702.60922287] 

[113.00037818 149.04859267 116.86268688 141.32397528] 

48.4164923678747


------------------

             x           y
8   517.218406  254.618364
9   549.404311  300.966068
10  554.554056  261.055545
11  518.505842  291.95401

In [ ]:
dir, subdir, pic, circle, dm, n_sub, dm_sub[n],freq_break_func[n-1], break_way='r'


In [62]:
pd.DataFrame({'a':[1,2],'b':[[1,1],[2,3,4]]})

,a,b
0,1,"[1, 1]"
1,2,"[2, 3, 4]"


In [ ]:


def tellme(s):
    print(s)
    plt.title(s, fontsize=16)
    plt.draw()

##################################################
# Define a triangle by clicking three points
##################################################
plt.clf()
plt.axis([-1., 1., -1., 1.])
plt.setp(plt.gca(), autoscale_on=False)

tellme('You will define a triangle, click to begin')

plt.waitforbuttonpress()

happy = False
while not happy:
    pts = []
    while len(pts) < 3:
        tellme('Select 3 corners with mouse')
        pts = np.asarray(plt.ginput(3, timeout=-1))
        if len(pts) < 3:
            tellme('Too few points, starting over')
            time.sleep(1)  # Wait a second

    ph = plt.fill(pts[:, 0], pts[:, 1], 'r', lw=2)

    tellme('Happy? Key click for yes, mouse click for no')

    happy = plt.waitforbuttonpress()

    # Get rid of fill
    if not happy:
        for p in ph:
            p.remove()

##################################################
# Now contour according to distance from triangle
# corners - just an example
##################################################


# Define a nice function of distance from individual pts
def f(x, y, pts):
    z = np.zeros_like(x)
    for p in pts:
        z = z + 1/(np.sqrt((x - p[0])**2 + (y - p[1])**2))
    return 1/z

X, Y = np.meshgrid(np.linspace(-1, 1, 51), np.linspace(-1, 1, 51))
Z = f(X, Y, pts)

CS = plt.contour(X, Y, Z, 20)

tellme('Use mouse to select contour label locations, middle button to finish')
CL = plt.clabel(CS, manual=True)

##################################################
# Now do a zoom
##################################################
tellme('Now do a nested zoom, click to begin')
plt.waitforbuttonpress()

happy = False
while not happy:
    tellme('Select two corners of zoom, middle mouse button to finish')
    pts = np.asarray(plt.ginput(2, timeout=-1))

    happy = len(pts) < 2
    if happy:
        break

    pts = np.sort(pts, axis=0)
    plt.axis(pts.T.ravel())

tellme('All Done!')
plt.show()

You will define a triangle, click to begin
Select 3 corners with mouse
Happy? Key click for yes, mouse click for no
Use mouse to select contour label locations, middle button to finish
Select label locations manually using first mouse button.
End manual selection with second mouse button.
Now do a nested zoom, click to begin
Select two corners of zoom, middle mouse button to finish
Select two corners of zoom, middle mouse button to finish
Select two corners of zoom, middle mouse button to finish
Select two corners of zoom, middle mouse button to finish
All Done!
